# Imitating DragonFruit AI Fire Detection

In [1]:
!pip install -q ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.2/645.2 kB 5.7 MB/s eta 0:00:00


In [14]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # Just Using medium Model

In [ ]:
!curl -L "https://universe.roboflow.com/ds/hAFeU0NbXl?key=8AZvSOeEZY" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [4]:
with open('/content/train/labels/large_-1138-_jpg.rf.a2d1a2684b58cf422915694e7d9ed004.txt', 'r') as file:
    for line in file:
        print(line, end='')

0 0.46274038461538464 0.40625 0.3894230769230769 0.3918269230769231
2 0.6658653846153846 0.13822115384615385 0.2932692307692308 0.2091346153846154
2 0.7728365384615384 0.4795673076923077 0.16706730769230768 0.49759615384615385

In [5]:
import os
import shutil
from random import shuffle

def move_files(src_files, src_labels, dest_path_images, dest_path_labels):
    os.makedirs(dest_path_images, exist_ok=True)
    os.makedirs(dest_path_labels, exist_ok=True)

    for src_file, src_label in zip(src_files, src_labels):
        # Define the destination files
        dest_file_image = os.path.join(dest_path_images, os.path.basename(src_file))
        dest_file_label = os.path.join(dest_path_labels, os.path.basename(src_label))

        # Move the files
        shutil.move(src_file, dest_file_image)
        shutil.move(src_label, dest_file_label)

# Define paths
train_images_path = '/content/train/images'
train_labels_path = '/content/train/labels'
val_images_path = '/content/val/images'
val_labels_path = '/content/val/labels'
test_images_path = '/content/test/images'
test_labels_path = '/content/test/labels'

# Get files
image_files = [os.path.join(train_images_path, f) for f in os.listdir(train_images_path) if os.path.isfile(os.path.join(train_images_path, f))]
label_files = [os.path.join(train_labels_path, f) for f in os.listdir(train_labels_path) if os.path.isfile(os.path.join(train_labels_path, f))]

# Make sure the files are in the same order so labels match the images
image_files.sort()
label_files.sort()

# Shuffle files
combined = list(zip(image_files, label_files))
shuffle(combined)
image_files[:], label_files[:] = zip(*combined)

# Calculate the number of files for validation and test sets
val_size = test_size = int(0.1 * len(image_files))

# Move 10% of images and labels to validation folders
move_files(image_files[:val_size], label_files[:val_size], val_images_path, val_labels_path)

# Move another 10% of images and labels to test folders
move_files(image_files[val_size:val_size+test_size], label_files[val_size:val_size+test_size], test_images_path, test_labels_path)

In [10]:
%%writefile data.yaml
train: /content/train/images
val: /content//val/images
test: /content//test/images

nc: 3
names: ['fire', 'other', 'smoke']

Overwriting data.yaml


In [15]:
results = model.train(data='data.yaml', epochs=50, imgsz=640)

Ultralytics YOLOv8.0.207 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=data.yaml, epochs=50, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8